# Huggingface Sagemaker-sdk - Deploy 🤗 Transformers for inference

1. [Introduction](#Introduction)  
    a. [Deploy the model directly after training](#Deploy-the-model-directly-after-training)  
    b. [Deploy the model using `model_data`](#Deploy-the-model-using-model_data)  

HuggingFace Inference DLCsとAmazon SageMaker Python SDKを使用して、Transformersモデルをデプロイします。    
このNotebookでは学習済みのHuggingFace TransformersモデルをSageMakerにデプロイして推論します。

## API - [SageMaker Hugging Face Inference Toolkit](https://github.com/aws/sagemaker-huggingface-inference-toolkit)


`transformers pipelines`を利用して、`pipelines`の全機能を簡単に利用できるAPIを設計しました。APIは[🤗 Accelerated Inference API](https://api-inference.huggingface.co/docs/python/html/detailed_parameters.html)のAPIを参考にしています。つまり、入力は `inputs` keyで定義する必要があり、サポートされている `pipelines` のパラメータを追加したい場合には `parameters` keyで追加することができます。以下にリクエストの例を示します。

**text-classification request body**
```python
{
	"inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}
```
**question-answering request body**
```python
{
	"inputs": {
		"question": "What is used for inference?",
		"context": "My Name is Philipp and I live in Nuremberg. This model is used with sagemaker for inference."
	}
}
```
**zero-shot classification request body**
```python
{
	"inputs": "Hi, I recently bought a device from your company but it is not working as advertised and I would like to get reimbursed!",
	"parameters": {
		"candidate_labels": [
			"refund",
			"legal",
			"faq"
		]
	}
}
```

## Deploy a Hugging Face Transformer model from S3 to SageMaker for inference

SageMakerで学習したHuggingFaceモデルをAmazon S3からデプロイするには2つの方法があります。トレーニングジョブ終了後に直接デプロイする方法と、S3に保存されたモデルを指す`model_data`を使って後からデプロイする方法があります。

### Deploy the model directly after training

トレーニングジョブ後にモデルを直接デプロイする場合は、トークナイザーやモデルなど、必要なファイルがトレーニングスクリプトに保存されていることを確認する必要があります。

_**NOTE: 日本語の場合、（トークナイザ周りで）追加ライブラリが必要になるため、後からデプロイのみに対応しています（トレーニングジョブ終了後に直接デプロイしたい場合はコンテナの作成が必要です）。**_

```python
from sagemaker.huggingface import HuggingFace

############ pseudo code start ############

# create HuggingFace estimator for running training
huggingface_estimator = HuggingFace(....)

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit(...)

############ pseudo code end ############

# deploy model to SageMaker Inference
predictor = huggingface_estimator.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

# example request, you always need to define "inputs"
data = {
   "inputs": "Camera - You are awarded a SiPix Digital Camera! call 09061221066 fromm landline. Delivery within 28 days."
}

# request
predictor.predict(data)

```

### Deploy the model using `model_data`

In [ ]:
!pip install --upgrade pip
!pip install "sagemaker>=2.48.1" "transformers[ja]==4.6.1" "datasets[s3]==1.6.2" --upgrade

In [ ]:
from sagemaker.huggingface import HuggingFaceModel
import sagemaker 

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()
sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
# S3に保存されたmodel_path
model_path = 'YOUR-MODEL-S3-PATH'

In [ ]:
from sagemaker.s3 import S3Downloader

# 学習したモデルのダウンロード
S3Downloader.download(
    s3_uri=model_path, # s3 uri where the trained model is located
    local_path='.', # local path where *.targ.gz is saved
    sagemaker_session=sess # sagemaker session used for training the model
)

In [ ]:
import os

# modelというフォルダがなければ作るコマンド
model_dir_path = './model/'

if not os.path.exists(model_dir_path):
    os.makedirs(model_dir_path)

In [ ]:
# ModelのUnzip
!tar -zxvf model.tar.gz -C model

Modelと共に日本語処理のためのトークナイザを保存したいため、ダウンロードし、保存します。

In [ ]:
from transformers import AutoTokenizer

tokenizer_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
tokenizer.save_pretrained('./model')

In [ ]:
# ModelとTokenizerの圧縮
!cd model/ && tar -zcvf ../model.tar.gz . && cd ..

In [ ]:
bucket = sess.default_bucket()
prefix = "samples/sequenceclassification/model"

In [ ]:
# (Tokenizerが一緒になった)Modelを再度Uploadします
upload_path = f's3://{bucket}/{prefix}'

new_model_path = sagemaker.s3.S3Uploader.upload(
    local_path='model.tar.gz', 
    desired_s3_uri=upload_path
)

print(new_model_path)

デフォルトのHuggingFaceコンテナは日本語処理に必要なライブラリ`fugashi`と`ipadic`がないため、`requirements.txt`経由でインストールします。    
推論はSageMaker HuggingFaceのデフォルトを使用します。    
`inference.py`は空ですが、`requirements.txt`の実行のために配置して、以下で指定する必要があります。

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    source_dir='./scripts',
    entry_point='inference.py',
    model_data=new_model_path,  # path to your trained sagemaker model
    role=role, # iam role with permissions to create an Endpoint
    transformers_version="4.6", # transformers version used
    pytorch_version="1.7", # pytorch version used
    py_version="py36", # python version of the DLC
)

In [ ]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

In [ ]:
# example request, you always need to define "inputs"
data = {
    "inputs": 'ハワイアンの心和む音楽の中、ちょっとシリアスなドラマが展開していきます。音楽の力ってすごいな、って思いました。'
    }

In [ ]:
# request
predictor.predict(data)

In [ ]:
# delete endpoint
predictor.delete_endpoint()